In [ ]:
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import from_origin
import warnings
warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)

In [ ]:
metadata = pd.read_csv("data/metadata.csv")

In [ ]:
def rotate_matrix_90(mat):
    return mat[::-1].T

In [ ]:
new_metadata = metadata.copy()
new_metadata = new_metadata.iloc[0:0]

for _ , row in metadata.iterrows():
    img_path = row["path"]+".tif"
    with rasterio.open(img_path) as src:
        data = src.read()
        x, y = row["coord_x"], row["coord_y"]
        profile = src.profile

        img = data[0]

        for i in range(4):
            # Write the rotated image
            rotated_path = f"{row['path']}_rot{i}"

            # Define the metadata for the image
            width, height = 64,64
            transform = from_origin(0, height, 1, 1)  # Change parameters based on your data

            # Create a rasterio dataset and write the image data
            with rasterio.open(rotated_path+".tif", 'w', driver='GTiff', height=height, width=width, count=1, dtype=str(img.dtype), crs='EPSG:4326', transform=transform) as dst:
                dst.write(img,1)

            new_row = {'date': row["date"],
                    'id_coord': f"{row['id_coord']}_rot{i}",
                    'plume': row["plume"],
                    'set': row["set"],
                        'lat': row["lat"],
                        'lon': row["lon"],
                        'coord_x': x,
                        'coord_y': y,
                        'path': rotated_path}
            img = rotate_matrix_90(img)
            x,y = y, 64-x
            new_metadata = new_metadata.append(new_row,ignore_index=True)

new_metadata

<ipython-input-6-c1e4a8a1d0e9>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_metadata = new_metadata.append(new_row,ignore_index=True)
<ipython-input-6-c1e4a8a1d0e9>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_metadata = new_metadata.append(new_row,ignore_index=True)
<ipython-input-6-c1e4a8a1d0e9>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_metadata = new_metadata.append(new_row,ignore_index=True)
<ipython-input-6-c1e4a8a1d0e9>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_metadata = new_metadata.append(new_row,ignore_index=True)
<ipython-input-6-c1e4a8a1d0e9>:36: FutureWarning: The frame.append metho

,date,id_coord,plume,set,lat,lon,coord_x,coord_y,path
0,20230223,id_6675_rot0,yes,train,31.528750,74.330625,24,47,images/plume/20230223_methane_mixing_ratio_id_...
1,20230223,id_6675_rot1,yes,train,31.528750,74.330625,47,40,images/plume/20230223_methane_mixing_ratio_id_...
2,20230223,id_6675_rot2,yes,train,31.528750,74.330625,40,17,images/plume/20230223_methane_mixing_ratio_id_...
3,20230223,id_6675_rot3,yes,train,31.528750,74.330625,17,24,images/plume/20230223_methane_mixing_ratio_id_...
4,20230103,id_2542_rot0,yes,train,35.538000,112.524000,42,37,images/plume/20230103_methane_mixing_ratio_id_...
...,...,...,...,...,...,...,...,...,...
1715,20230213,id_5510_rot3,no,train,32.713854,44.609398,10,55,images/no_plume/20230213_methane_mixing_ratio_...
1716,20230330,id_6609_rot0,no,train,47.758979,27.801630,21,15,images/no_plume/20230330_methane_mixing_ratio_...
1717,20230330,id_6609_rot1,no,train,47.758979,27.801630,15,43,images/no_plume/20230330_methane_mixing_ratio_...
1718,20230330,id_6609_rot2,no,train,47.758979,27.801630,43,49,images/no_plume/20230330_methane_mixing_ratio_...


In [ ]:
new_metadata.to_csv("new_metadata.csv")